In [ ]:
import numpy as np
import pandas as pd
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
import re 
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import plotly
from sklearn.preprocessing import StandardScaler, scale
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
from dm_test2 import dm_test

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(y_true - y_pred))

## Подготовка данных

In [ ]:
tab_2014_1 = pd.read_excel('Macro data.xlsx', sheet_name = 'Сырые данные')
tab_2014_ = pd.read_excel('Macro data.xlsx', sheet_name = 'Данные')

print(tab_2014_.head(3))
print(tab_2014_.tail(3))
scaler = MinMaxScaler(feature_range=(0, 100))
scaler = scaler.fit(tab_2014_.iloc[:,1:])
try1 = scaler.transform(tab_2014_.iloc[:,1:])
tab_2014__ = pd.DataFrame(try1, columns=tab_2014_.iloc[:,1:].columns)

var_name = 'nonfood_goods' ## определяем целевую переменную

tab_2014 = tab_2014__.filter(['ipd', var_name, 'oil', 'real_rate', 'rvi'], axis = 1)
for i in tab_2014.columns: 
    tab_2014[i+'-1'] = tab_2014[i].shift().reset_index(drop = True)
    tab_2014[i+'-2'] = tab_2014[i+'-1'].shift().reset_index(drop = True)
    tab_2014[i+'-3'] = tab_2014[i+'-2'].shift().reset_index(drop = True) 
   # tab_2014[i+'-4'] = tab_2014[i+'-3'].shift().reset_index(drop = True)
   # tab_2014[i+'-5'] = tab_2014[i+'-4'].shift().reset_index(drop = True) 
   # tab_2014[i+'-6'] = tab_2014[i+'-5'].shift().reset_index(drop = True) 
len(tab_2014)

In [ ]:
tab_2014_12 = tab_2014_1.filter(['Продовольственные', 'Непродовольственные', 'Российский индекс волатильности',
                                 'Цена на нефть (BRENT)', 'Реальный валютный курс', 'Индекс потребительского доверия'])[:-4]
plotly.offline.init_notebook_mode()

for i in range(len(tab_2014_12.columns)):
    data=(plotly.graph_objs.Scatter(x=tab_2014_1.Дата, 
                                      y=pd.DataFrame(tab_2014_12).iloc[:,i], 
                                      mode='lines'))

    layout = go.Layout(legend=dict(orientation="h"),font=dict(size=15,),autosize=False, width=1000,height=700,
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
        yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
        linecolor = '#636363', linewidth = 1, showticklabels = True))

    fig = go.Figure(data=data, layout = layout)

    plotly.offline.iplot(fig) 

In [ ]:
prod = pd.read_excel('prod.xlsx')

prod.index = prod['Date']
prod = prod.drop(['Date'], axis = 1)
print(prod.tail(3))

prod_detrend = np.log(prod).diff(12)
X1 = prod_detrend[prod_detrend.index>='2015-01-01']

scaler = MinMaxScaler(feature_range
                      =(0, 100))
scaler = scaler.fit(X1)
try1 = scaler.transform(X1)
prod_detrend = pd.DataFrame(try1, columns=prod.columns)
pd.DataFrame(prod).describe().round(2)

In [ ]:
plotly.offline.init_notebook_mode()
data = []
for i in range(len(prod.columns)):
    data.append(plotly.graph_objs.Scatter(x=prod.index[132:], 
                                      y=prod.iloc[:,i][132:], 
                                      mode='lines', name = prod.columns[i]))
    
layout = go.Layout(font=dict(size=18,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

In [ ]:
neprod = pd.read_excel('neprod.xlsx')
neprod.index = neprod['Date']
neprod = neprod.drop(['Date'], axis = 1)
neprod.head(3)

neprod_detrend = np.log(neprod).diff(12)
X1 = neprod_detrend[neprod_detrend.index>='2015-01-01']

scaler = MinMaxScaler(feature_range=(0, 100))
scaler = scaler.fit(X1)
try1 = scaler.transform(X1)
neprod_detrend = pd.DataFrame(try1, columns=neprod.columns)
pd.DataFrame(neprod_detrend).describe().round(2)

In [ ]:
plotly.offline.init_notebook_mode()
data = []
for i in range(len(neprod.columns)):
    data.append(plotly.graph_objs.Scatter(x=neprod.index[132:], 
                                      y=pd.DataFrame(neprod).iloc[:,i][132:], 
                                      mode='lines', name = neprod.columns[i]))

layout = go.Layout(font=dict(size=18,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

## Тесты Дики-Фуллера

In [ ]:
prod_adj = pd.read_excel('prod_adj.xlsx') ## Для теста Дики-Фуллера
prod_adj.index = prod_adj['Dates']
prod_adj = prod_adj.drop(['Dates'], axis = 1)
series = prod_adj[prod_adj.index>='2014-01-01']

### H0 : ряд нестационарный. Н0 отвергается при p-value < 0.05.

results = []
for i in series.columns:
    X = series[i].values
    result = adfuller(X)
    results.append(round(result[1],2))
test_p = pd.DataFrame([series.columns, results])
test_p

In [ ]:
neprod_adj = pd.read_excel('neprod_adj.xlsx') ## Для теста Дики-Фуллера
neprod_adj.index = neprod_adj['Dates']
neprod_adj = neprod_adj.drop(['Dates'], axis = 1)
series =neprod_adj[neprod_adj.index>='2014-01-01']

### H0 : ряд нестационарный. Н0 отвергается при p-value < 0.05. 
results = []
for i in series.columns:
    X = series[i].values
    result = adfuller(X)
    results.append(round(result[1],2))
test_np = pd.DataFrame([series.columns, results])
test_np